<a href="https://colab.research.google.com/github/palkop11/soh-ml/blob/master/interaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparations

## Mount
Mount google drive so you can upload DATA from drive to colab environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Clone, change directory
Clone repository (works when repo is public), then jump to repo directory

In [ ]:
# clone repo
!git clone https://github.com/palkop11/soh-ml.git

# jump to repo directory
%cd soh-ml

fatal: destination path 'soh-ml' already exists and is not an empty directory.
/content/soh-ml


## Upload dataset
Before running cell below, make sure that 'dataset_v5_ts_npz.zip' was uploaded anywhere in your drive, and drive was mounted. \
Find archive and extract into content/soh-ml/DATA :

In [ ]:
# find "datset_v5_ts_npz.zip" and then extract it into temporary workspace
!find /content/drive -name "dataset_v5_ts_npz.zip" | xargs -I {} unzip -qo {} -d "./DATA/"
# check if dataset directory was created
!find /content/soh-ml/DATA -maxdepth 1

/content/soh-ml/DATA
/content/soh-ml/DATA/dataset_v5_ts_npz


## Installs, imports, fix seed

Run cell below to install packages

In [ ]:
"""
here should be pip install from requirements_colab for reproducibility
but now it results in errors :/
!pip install -r requirements_colab.txt -q
"""

!pip install tensorboard tbparse pytorch-lightning lightning -q

make sure that you are in project directory content/soh-ml, otherwise some imports will fail

In [ ]:
%cd /content/soh-ml/

/content/soh-ml


make imports, fix seed

In [ ]:
import learning
import cross_validation

from lightning import seed_everything
seed_everything(42)

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42


42

# Running experiments

## how to run single experiment

you can use run_experiment function from learning.py to run experiment. \
This will run experiment with default testing configuration:
```python
learning.run_experiment(learning.test_config)
```
Also, you can run it from command line. Without arguments it will run with test_config configuration:
```bash
!python learning.py
```

To run experiment with your own config, you can specify it using python code and then pass config dictionary as argument to run_experiment function:
```python
# all keys (except ['test'] in ['data'])
# shoud be present in this single experiment configure dictionary:
my_config = {
    'experiment_name': 'my_experiment', # also used for TensorBoard logging
    'seed': 42,

    'data': {
        'datadir':'./DATA/dataset_v5_ts_npz/',
        # 'supported subsets as string arguments:
        # 'blacklist', 'small', 'train', 'val', 'test'
        # or you can specify subset using list with battery IDs
        'train': 'train',
        'val': 'val',
        #'test': None # normally, you dont need to specify test dataset
        # supported normalization types:
        # 'minmax_zero_one', 'minmax_symmetric', 'meanimax', 'meanstd'
        'normalization': {'x': None, 'y': 'minmax_zero_one'},
        'n_diff': 0,
    },

    'model': {
        'input_size': 2,
        'cnn_hidden_dim': 32,
        'cnn_channels': [4, 8, 16],
        'lstm_hidden_size': 32,
        'num_layers': 1,
        'output_size': 1, # do not change that
        'dropout': 0.,
        'regressor_hidden_dim': 1024,
        'output_activation': 'sigmoid', # 'tanh', 'sigmoid', 'relu' are supported
    },

    'training': {
        'resume_ckpt': None,
        'batch_size': 32,
        'learning_rate': 1e-3,
        'loss_type': 'huber', # 'mse', 'huber', 'bce' are supported
        'epochs': 1,
        'accelerator': 'auto',
        'devices': 1,
    },

    'metrics': 'all', # 'all', 'mse', 'mae', 'mape', 'r2', 'pcc' are supported

    'logging': {
        'log_dir': './LOGS',
        'progress_bar': True,
    }
```
and then run experiment just like this:
```python
learning.run_experiment(my_config)
```
also, such config may be stored in .yaml format and then you can pass it to run_experiment:
```python
learning.run_experiment('path/to/my/config/my_config.yaml')
```
also you can specify config when running learning.py in command line:
```bash
!python learning.py --config path/to/my/config.yaml
```

during experiment, inside LOGS directory (or any other 'log_dir', which you will specify in config) \
would be created directory for each experiment and version: \
my_experiment/version_0